In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Step 1: संरचित आउटपुट के लिए Pydantic मॉडल्स को परिभाषित करें

ये मॉडल्स वह **स्कीमा** परिभाषित करते हैं जिसे एजेंट्स लौटाएंगे। Pydantic के साथ `response_format` का उपयोग करने से सुनिश्चित होता है:
- ✅ टाइप-सुरक्षित डेटा निष्कर्षण
- ✅ स्वचालित सत्यापन
- ✅ मुक्त-पाठ प्रतिक्रियाओं से कोई पार्सिंग त्रुटि नहीं
- ✅ फ़ील्ड्स के आधार पर सरल सांकेतिक मार्गनिर्देशन


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Step 2: होटल बुकिंग टूल बनाना

यह टूल वह है जिसे **availability_agent** कॉल करेगा यह जांचने के लिए कि कमरे उपलब्ध हैं या नहीं। हम `@ai_function` डेकोरेटर का उपयोग करते हैं ताकि:
- एक पायथन फंक्शन को AI-कॉल करने योग्य टूल में परिवर्तित किया जा सके
- LLM के लिए स्वचालित रूप से JSON स्कीमा जनरेट किया जा सके
- पैरामीटर सत्यापन को संभाला जा सके
- एजेंट्स द्वारा स्वचालित आह्वान सक्षम किया जा सके

इस डेमो के लिए:
- **स्टॉकहोम, सिएटल, टोक्यो, लंदन, एम्स्टर्डम** → कमरे उपलब्ध हैं ✅
- **अन्य सभी शहर** → कोई कमरे नहीं ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Step 3: रूटिंग के लिए कंडीशन फंक्शंस परिभाषित करें

ये फंक्शंस एजेंट के प्रतिक्रिया की जांच करते हैं और निर्धारित करते हैं कि वर्कफ़्लो में कौन सा मार्ग लेना है।

**मुख्य पैटर्न:**
1. जांचें कि संदेश `AgentExecutorResponse` है या नहीं
2. संरचित आउटपुट (Pydantic मॉडल) पार्स करें
3. रूटिंग को नियंत्रित करने के लिए `True` या `False` लौटाएं

वर्कफ़्लो इन कंडीशंस का **एजेस** पर मूल्यांकन करेगा यह निर्णय लेने के लिए कि अगला कौन सा एक्जीक्यूटर सक्रिय किया जाए।


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Step 4: कस्टम डिस्प्ले एक्जीक्यूटर बनाएं

एक्जीक्यूटर्स वर्कफ़्लो घटक होते हैं जो ट्रांसफॉर्मेशन या साइड इफेक्ट्स को अंजाम देते हैं। हम अंतिम परिणाम दिखाने के लिए `@executor` डेकोरेटर का उपयोग करके एक कस्टम एक्जीक्यूटर बनाते हैं।

**प्रमुख अवधारणाएं:**
- `@executor(id="...")` - एक फंक्शन को वर्कफ़्लो एक्जीक्यूटर के रूप में पंजीकृत करता है
- `WorkflowContext[Never, str]` - इनपुट/आउटपुट के लिए टाइप संकेत
- `ctx.yield_output(...)` - अंतिम वर्कफ़्लो परिणाम देता है


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Step 5: पर्यावरण चर लोड करें

LLM क्लाइंट को कॉन्फ़िगर करें। यह उदाहरण निम्न के साथ काम करता है:
- **GitHub Models** (GitHub टोकन के साथ मुफ्त स्तर)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## चरण 6: संरचित आउटपुट वाले AI एजेंट बनाएं

हम **तीन विशेषज्ञ एजेंट** बनाते हैं, जिनमें से प्रत्येक को `AgentExecutor` में लपेटा गया है:

1. **availability_agent** - टूल का उपयोग करके होटल की उपलब्धता जांचता है
2. **alternative_agent** - वैकल्पिक शहर सुझाता है (जब कमरे न हों)
3. **booking_agent** - बुकिंग के लिए प्रोत्साहित करता है (जब कमरे उपलब्ध हों)

**मुख्य विशेषताएँ:**
- `tools=[hotel_booking]` - एजेंट को टूल प्रदान करता है
- `response_format=PydanticModel` - संरचित JSON आउटपुट को मजबूर करता है
- `AgentExecutor(..., id="...")` - वर्कफ़्लो उपयोग के लिए एजेंट को लपेटता है


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## चरण 7: कंडीशनल एज के साथ वर्कफ़्लो बनाएं

अब हम `WorkflowBuilder` का उपयोग करके कंडीशनल रूटिंग के साथ ग्राफ़ बनाते हैं:

**वर्कफ़्लो संरचना:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**मुख्य विधियाँ:**
- `.set_start_executor(...)` - प्रवेश बिंदु सेट करता है
- `.add_edge(from, to, condition=...)` - कंडीशनल एज जोड़ता है
- `.build()` - वर्कफ़्लो को अंतिम रूप देता है


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Step 8: टेस्ट केस 1 चलाएं - शहर बिना उपलब्धता के (पैरिस)

आइए **कोई उपलब्धता नहीं** पथ का परीक्षण करें पैरिस में होटल्स अनुरोध करके (जिसके हमारे सिमुलेशन में कोई कमरे नहीं हैं)।


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Step 9: टेस्ट केस 2 चलाएं - शहर जिसमें उपलब्धता हो (स्टॉकहोम)

अब चलिए **उपलब्धता** पथ का परीक्षण करते हैं स्टॉकहोम में होटलों का अनुरोध करके (जिसके पास हमारे सिमुलेशन में कमरे हैं)।


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## मुख्य निष्कर्ष और अगले कदम

### ✅ आपने क्या सीखा:

1. **WorkflowBuilder पैटर्न**
   - प्रवेश बिंदु परिभाषित करने के लिए `.set_start_executor()` का उपयोग करें
   - सशर्त मार्गदर्शन के लिए `.add_edge(from, to, condition=...)` का उपयोग करें
   - वर्कफ़्लो को अंतिम रूप देने के लिए `.build()` कॉल करें

2. **सशर्त मार्गदर्शन**
   - कंडीशन फंक्शन `AgentExecutorResponse` का निरीक्षण करते हैं
   - मार्ग निर्धारण के फैसलों के लिए संरचित आउटपुट पार्स करें
   - किसी एज को सक्रिय करने के लिए `True` लौटाएं, छोड़ने के लिए `False`

3. **टूल इंटिग्रेशन**
   - पायथन फंक्शन को AI टूल में बदलने के लिए `@ai_function` का उपयोग करें
   - एजेंट जरूरत पड़ने पर टूल्स को स्वचालित रूप से कॉल करते हैं
   - टूल्स JSON लौटाते हैं जिसे एजेंट पार्स कर सकते हैं

4. **संरचित आउटपुट**
   - प्रकार-सुरक्षित डेटा निष्कर्षण के लिए Pydantic मॉडल का उपयोग करें
   - एजेंट बनाते समय `response_format=MyModel` सेट करें
   - `Model.model_validate_json()` के साथ प्रतिक्रियाओं को पार्स करें

5. **कस्टम एग्जीक्यूटर्स**
   - वर्कफ़्लो कॉम्पोनेंट बनाने के लिए `@executor(id="...")` का उपयोग करें
   - एग्जीक्यूटर्स डेटा को ट्रांसफॉर्म कर सकते हैं या साइड इफ़ेक्ट कर सकते हैं
   - वर्कफ़्लो परिणाम उत्पन्न करने के लिए `ctx.yield_output()` का उपयोग करें

### 🚀 वास्तविक दुनिया में उपयोग:

- **ट्रैवल बुकिंग**: उपलब्धता चेक करें, विकल्प सुझाएं, तुलना करें
- **कस्टमर सर्विस**: समस्या के प्रकार, भावना, प्राथमिकता के आधार पर रूटिंग
- **ई-कॉमर्स**: इन्वेंट्री चेक करें, विकल्प सुझाएं, ऑर्डर प्रोसेस करें
- **कंटेंट मॉडरेशन**: टॉक्सिसिटी स्कोर, यूजर फ्लैग के आधार पर रूटिंग
- **अनुमोदन वर्कफ़्लोज़**: राशि, यूजर रोल, जोखिम स्तर के आधार पर रूटिंग
- **मल्टी-स्टेज प्रोसेसिंग**: डेटा गुणवत्ता, पूर्णता के आधार पर रूटिंग

### 📚 अगले कदम:

- अधिक जटिल कंडीशन जोड़ें (कई मानदंड)
- वर्कफ़्लो स्टेट प्रबंधन के साथ लूप्स लागू करें
- पुन: उपयोग के लिए सब-वर्कफ़्लोज़ जोड़ें
- वास्तविक API (होटल बुकिंग, इन्वेंट्री सिस्टम) के साथ इंटीग्रेट करें
- त्रुटि हैंडलिंग और फॉलबैक पथ जोड़ें
- बिल्ट-इन विज़ुअलाइज़ेशन टूल्स के साथ वर्कफ़्लोज़ विज़ुअलाइज़ करें


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता के लिए प्रयासरत हैं, कृपया ध्यान दें कि स्वचालित अनुवादों में त्रुटियाँ या असंगतियाँ हो सकती हैं। मूल दस्तावेज़ को उसकी मूल भाषा में प्रमाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए पेशेवर मानव अनुवाद की सलाह दी जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम जिम्मेदार नहीं हैं।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
